Make master file

In [3]:
import os

# Define the root directory
root_directory = "/n17data/tersenov/Cov_DES_SLICS"

# Open the master text file in write mode
master_file_path = ".././input/master_file_cov.txt"
with open(master_file_path, "w") as master_file:
    # Iterate over the subdirectories matching the pattern "LOS..."
    for subdir in os.listdir(root_directory):
        # Check if the subdirectory name matches the pattern "LOS..."
        if subdir.startswith("LOS") and subdir[3:].isdigit():
            subdir_path = os.path.join(root_directory, subdir)
            
            # Iterate over the files in the subdirectory
            for file_name in os.listdir(subdir_path):
                # Check if the file name matches the desired pattern
                if file_name.startswith("DES_MocksCat_SLICS_4_Bin") and file_name.endswith(".dat"):
                    file_path = os.path.join(subdir_path, file_name)
                    master_file.write(file_path + "\n")


In [7]:
import numpy as np

def parse_cov_SLICS_filenames(file_paths):
    # Make empty recarray to store the data
    data = np.recarray(len(file_paths), dtype=[('bin', int), ('LOS', int), ('tile', int)])

    # Iterate over the file paths and process each file
    for i, file_path in enumerate(file_paths):
        # Extract the file name from the file path
        file_name = file_path.split("/")[-1]
        
        # Split file name into parts
        file_parts = file_name.split("_")
        
        # Extract relevant information
        bin = int(file_parts[4][3:])  # Extract the number after "Bin"
        LOS = int(file_parts[5][3:])  # Extract the number after "LOS"
        tile = int(file_parts[6][1:-4])  # Extract the number after "R"

        # Assign the extracted data to the corresponding fields in the recarray
        data[i]['bin'] = bin
        data[i]['LOS'] = LOS
        data[i]['tile'] = tile

    return data


In [ ]:
# Read the file paths from master_file.txt
filename = ".././input/master_file.txt"
with open(filename, 'r') as file:
    file_paths = file.readlines()
    file_paths = [path.strip() for path in file_paths]

filename_data = slics.parse_SLICS_filenames(file_paths)

In [8]:
# Define the path to the "master_file_cov.txt"
master_file_path = ".././input/master_file_cov.txt"

# Read the file paths from the "master_file_cov.txt"
with open(master_file_path, "r") as file:
    file_paths = file.readlines()
    file_paths = [path.strip() for path in file_paths]

# Now, you can call the modified parse_SLICS_filenames function to parse these file paths
parsed_cov_data = parse_cov_SLICS_filenames(file_paths)


In [9]:
parsed_cov_data

rec.array([(1, 100,  1), (1, 100,  2), (1, 100,  3), ..., (4,  99, 17),
           (4,  99, 18), (4,  99, 19)],
          dtype=[('bin', '<i8'), ('LOS', '<i8'), ('tile', '<i8')])